In [ ]:
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd
import pygsheets
import webbrowser
from tkinter import *
from tkinter import messagebox as tkMessageBox
from tkinter import filedialog
import threading
class GUI_python:
    def gui_guide(self):
        self.frame = Tk()
        self.frame.resizable(0, 0)
        self.frame.title("HDFC AMC AUM/Statewise Data")
        self.frame.geometry('900x200')
        # Above few lines of codes create a window
        self.emailFrame = Frame(self.frame, width=200, height=600, highlightthickness=2,
                                highlightbackground="#111")
        # This creates a frame which creates a box around label which has enter email id text
        self.emailFrame.grid(row=1,column=0, padx=10, pady=2, sticky=N + S)
        self.servicFrame = Frame(self.frame, width=200, height=300, highlightthickness=2,
                                  highlightbackground="#111")
        self.servicFrame.grid(row=2, column=0, padx=10, pady=2, sticky=N + S)
        self.driverFrame = Frame(self.frame, width=200, height=300, highlightthickness=2,
                                 highlightbackground="#111")
        self.driverFrame.grid(row=3, column=0, padx=10, pady=2, sticky=N + S)
        self.l3= Label(self.servicFrame, text="Upload the google API service credentials:")
        self.l3.grid(row=2, column=0)
        self.l1 = Label(self.driverFrame, text="Upload Chromedriver:")
        self.l1.grid(row=3, column=0)
        self.l4=Label(self.emailFrame, text="Enter your mail id:")
        self.l4.grid(row=1, column=0)
        self.l5= Label(self.frame, text="No File Selected",bg = "white", pady = 5)
        self.l5.grid(row=2, column=1)
        self.l7 = Label(self.frame, text="No File Selected", bg="white", pady=5)
        self.l7.grid(row=3, column=1)
        self.E1 = Entry(self.frame, bd=5)
        self.E1.grid(row=1, column=1)
        self.btn2= Button(self.frame, text="Submit", command=self.sci_thread)
        self.btn2.grid(row=5, column=0)
        self.btn3 = Button(self.frame, text="Close", command=self.frame.destroy)
        self.btn3.grid(row=5, column=1)
        self.btn1 = Button(self.frame, text='Browse Credentials', command=self.browse_button)
        self.btn1.grid(row=2, column=2)
        self.btn4 = Button(self.frame, text='Browse Driver', command=self.browse_button_1)
        self.btn4.grid(row=3, column=2)
        self.l6 = Label(self.frame, text="Click Submit to Update",bg = "white")
        self.l6.grid(row=6,column=0)
        self.frame.mainloop()
    def proces(self):
        # Verify that the data entered is correct or not before submitting
        try:
            global email
            email=str(Entry.get(self.E1))
            text=self.l5.cget("text")
            text2=self.l7.cget("text")
            # When no text is entered in any entry and no file is selected raise this warning
            if text2=="No File Selected" or email==""or text=="No File Selected" or("@" not in email):
                if email=="" or "@" not in email:
                    tkMessageBox.showwarning("Warning", "Please enter a valid Ticker, API Key and Email id")
                if "No File" in text:
                    tkMessageBox.showwarning("Warning", "Please select google credentials file")
                if "No File" in text2:
                    tkMessageBox.showwarning("Warning", "Please select the Chromedriver")
            else:
                self.scraping()
                self.l6.config(text="Data Updated")
                # This calls the google sheets function
        except:
            tkMessageBox.showwarning("Warning", "Please enter Valid data")

    def sci_thread(self):
        maincal = threading.Thread(target=self.proces)
        maincal.start()
    def browse_button(self):
        # This function act as submit button for google credentials
        filename1 = filedialog.askopenfilename(initialdir="/", title="Select file",filetypes=(("json files", "*.json"), ("all files", "*.*")))
        if filename1:
            self.l5.config(text=filename1)
    def browse_button_1(self):
        # This function act as submit button for chromedriver
        filename1 = filedialog.askopenfilename(initialdir="/", title="Select file",filetypes=(("exe files", "*.exe"), ("all files", "*.*")))
        if filename1:
            self.l7.config(text=filename1)
    def scraping(self):
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)
        driver = Chrome(executable_path=self.l7.cget("text"),options=chrome_options)
        # The line of code below calls the webdriver and opens this website
        driver.get('https://www.hdfcfund.com/statutory-disclosure/aum')
        time.sleep(3)
        wait = WebDriverWait(driver,15)
        click_element=wait.until(EC.element_to_be_clickable((By.XPATH,"//*[@id='app']/div/div[3]/div[1]/div/div/div[2]/button[1]")))
        click_element.click()
        dropdown=wait.until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/div/div[2]/div/div/div[3]/div[1]/div/div/div")))
        dropdown.click()
        monthly=wait.until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/div/div[2]/div/div/div[3]/div[1]/div/div/div/ul/div/li[1]")))
        monthly.click()
        #page_view=wait.until(EC.presence_of_all_elements_located((By.XPATH,'/html/body/div[1]/div/div[2]/div/div/div[3]/div[3]/div/span')))
        data=wait.until(EC.presence_of_element_located((By.XPATH,f'/html/body/div[1]/div/div[2]/div/div/div[3]/div[2]/div/div[1]/a')))
        name=data.text
        print(name)
        link=data.get_attribute('href')
        year=re.sub("[^0-9]", "",data.text)
        gc = pygsheets.authorize(service_file=self.l5.cget("text"))
        driver.quit()
        temp = pd.ExcelFile(link)
        aum_data = pd.read_excel(temp, sheet_name=0)
        aum_data.fillna(" ", inplace=True)
        city_data = pd.read_excel(temp, sheet_name=1)
        city_data.fillna(" ", inplace=True)
        try:
            # This try to call the google sheets if present
            aum= gc.open(f'HDFC AMC Net AUM Scheme-wise for {year}')
            #
            city_sheet=gc.open(f'HDFC AMC Statewise Contribution to AUM for {year}')
            try:
                wks = aum.worksheet_by_title(name)
                wks2=city_sheet.worksheet_by_title(name)
                print("done")
                webbrowser.open(aum.url,new=2)
                webbrowser.open(city_sheet.url,new=2)
            except:
                print("done1")
                aum.share(str(Entry.get(self.E1)), role='writer', type='user')
                city_sheet.share(str(Entry.get(self.E1)), role='writer', type='user')
                aum.add_worksheet(name)
                wks= aum.worksheet_by_title(name)
                wks.set_dataframe(aum_data, start='A1')
                city_sheet.add_worksheet(name)
                wks= city_sheet.worksheet_by_title(name)
                wks.set_dataframe(city_data, start='A1')
                webbrowser.open(aum.url, new=2)
                webbrowser.open(city_sheet.url, new=2)
        except:
            print("done2")
            aum = gc.sheet.create(f'HDFC AMC Net AUM Scheme-wise for {year}')
            aum = gc.open_by_key(aum['spreadsheetId'])
            aum.share(str(Entry.get(self.E1)), role='writer', type='user')
            aum.sheet1.title=name
            wks=aum.sheet1
            wks.set_dataframe(aum_data, start='A1')
            city_sheet= gc.sheet.create(f'HDFC AMC Statewise Contribution to AUM for {year}')
            city_sheet= gc.open_by_key(city_sheet['spreadsheetId'])
            city_sheet.share(str(Entry.get(self.E1)), role='writer', type='user')
            city_sheet.sheet1.title=name
            wks=city_sheet.sheet1
            wks.set_dataframe(city_data, start='A1')

def main():
    gui = GUI_python()
    tar = gui.gui_guide()
if __name__ == "__main__":
    main()








